# CSE 541 HW 3
Evan Komp

In [12]:
import numpy as np
from mnist import MNIST
import sklearn.preprocessing
import sklearn.decomposition

***
## 2
Load and contextualize the data

In [35]:
mndata = MNIST('./data')
# mndata.select_emnist('balanced')
images, labels = mndata.load_training('balanced')

TypeError: MNIST.load_training() takes 1 positional argument but 2 were given

In [26]:
images = np.array(images)
y = np.array(labels)

In [27]:
#normalize images
C = images/255

In [29]:
# PCA to d
d = 10
pca = sklearn.decomposition.PCA(10)
C = pca.fit_transform(C)

In [30]:
# function to map context and action to features
n=10
def phi(c, i, n=n):
    out = np.zeros((n, c.size))
    out[i] = c
    return out.flatten()

In [33]:
np.unique(y, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]))